# DataSet e bibliotecas a serem usados no projeto

In [46]:
import pandas as pd
import jax.numpy as jnp
import numpy as np
import jax as jax
from typing import Callable
from sklearn.model_selection import train_test_split
import time
import timeit
import matplotlib.pyplot as plt

# user id | item id | rating | timestamp.
_df = pd.read_csv('ml-100k/u.data', delimiter='\t', header=None, names=['userId', 'movieId', 'rating', 'timeStamp'])
originalData = _df.pivot(index='userId', columns='movieId', values='rating')
print(_df.head(), "\n", originalData.shape)


   userId  movieId  rating  timeStamp
0     196      242       3  881250949
1     186      302       3  891717742
2      22      377       1  878887116
3     244       51       2  880606923
4     166      346       1  886397596 
 (943, 1682)


### Separação de dados de teste

De acordo com a documentação, os dados já estão ordenados de maneira aleatória, então podemos apenas pegar os primeiros 20% de entradas.

In [47]:
testData = _df[:int(0.2*len(_df))]
_df = _df[int(0.2*len(_df)):]

# Tratamento dos dados: transposição em matriz, preenchimento de dados faltantes, normalização

In [48]:
# Convertendo a lista de dados em uma tabela com usuários nas linhas, filmes nas colunas, contendo os ratings correspondentes.
trainData = _df.pivot(index='userId', columns='movieId', values='rating')
testData = testData.pivot(index='userId', columns='movieId', values='rating')

# # Reindexando os dataframes separados para incluuir todos os userIds e movieIds do dataframe original
# userIds = [_ for _ in range(1, 944)]
# movieIds = [_ for _ in range(1, 1683)]

# trainData = trainData.reindex(index=userIds, columns=movieIds)
# testData = testData.reindex(index=userIds, columns=movieIds)

trainData = trainData.iloc[:10, :10]
trainData


movieId,1,2,3,4,5,6,7,8,9,10
userId,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,NaN,4.0,1.0,5.0,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,4.0,NaN,NaN,NaN,NaN,NaN,2.0,4.0,4.0,NaN
7,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,5.0,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN


In [49]:
# Preenchendo valores faltantes com o rating médio do filme correspondente
trainData = trainData.apply(
    lambda x: x.fillna(0) if x.isna().all() else x.fillna(x.mean(numeric_only=True)), axis=0)

trainData

movieId,1,2,3,4,5,6,7,8,9,10
userId,,,,,,,,,,
1,5.000000,3.0,4.0,3.0,3.0,0.0,4.000000,1.0,5.0,2.0
2,4.000000,3.0,4.0,4.0,3.0,0.0,3.333333,2.5,4.5,2.0
3,4.333333,3.0,4.0,4.0,3.0,0.0,3.333333,2.5,4.5,2.0
4,4.333333,3.0,4.0,4.0,3.0,0.0,3.333333,2.5,4.5,2.0
5,4.333333,3.0,4.0,4.0,3.0,0.0,3.333333,2.5,4.5,2.0
6,4.000000,3.0,4.0,4.0,3.0,0.0,2.000000,4.0,4.0,2.0
7,4.333333,3.0,4.0,5.0,3.0,0.0,3.333333,2.5,5.0,2.0
8,4.333333,3.0,4.0,4.0,3.0,0.0,3.333333,2.5,4.5,2.0
9,4.333333,3.0,4.0,4.0,3.0,0.0,4.000000,2.5,4.5,2.0


In [50]:
trainData = jnp.array(trainData.values)
trainData

Array([[5.       , 3.       , 4.       , 3.       , 3.       , 0.       ,
        4.       , 1.       , 5.       , 2.       ],
       [4.       , 3.       , 4.       , 4.       , 3.       , 0.       ,
        3.3333333, 2.5      , 4.5      , 2.       ],
       [4.3333335, 3.       , 4.       , 4.       , 3.       , 0.       ,
        3.3333333, 2.5      , 4.5      , 2.       ],
       [4.3333335, 3.       , 4.       , 4.       , 3.       , 0.       ,
        3.3333333, 2.5      , 4.5      , 2.       ],
       [4.3333335, 3.       , 4.       , 4.       , 3.       , 0.       ,
        3.3333333, 2.5      , 4.5      , 2.       ],
       [4.       , 3.       , 4.       , 4.       , 3.       , 0.       ,
        2.       , 4.       , 4.       , 2.       ],
       [4.3333335, 3.       , 4.       , 5.       , 3.       , 0.       ,
        3.3333333, 2.5      , 5.       , 2.       ],
       [4.3333335, 3.       , 4.       , 4.       , 3.       , 0.       ,
        3.3333333, 2.5      , 4.5    

In [51]:

# Normalizando os dados: subtração da média de cada feature e mapeando para o intervalo [0,1]
# Em alguns casos std era zero, e portanto um pequena constante é adicionada aos valroes para evitar divisão por zero
mean = jnp.mean(trainData, axis=0)
std = jnp.std(trainData, axis=0) + 1e-12

normalizedTrainData = trainData - mean
# normalizedTrainData = normalizedTrainData / std

normalizedTrainData

Array([[ 0.6666665 ,  0.        ,  0.        , -1.        ,  0.        ,
         0.        ,  0.66666675, -1.5       ,  0.5       ,  0.        ],
       [-0.3333335 ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [-0.3333335 ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        , -1.3333333 ,  1.5       , -0.5       ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ,  0.        ,  0.        

# Definindo as funções para implementação do SVD, KNN e PCA

In [52]:
def reorder(eigenvalues:jnp.ndarray, eigenvectors:jnp.ndarray) -> tuple[jnp.ndarray, jnp.ndarray]:
    """ Reorders in descending order the eigenvalues and corresponding eigenvectors based on the eigenvectors values.

    Args:
        eigenvalues (jnp.ndarray): 1D array containing the eigenvalues.
        eigenvectors (jnp.ndarray): 2D array cotaining eigenvectors as columns.

    Returns:
        tuple[jnp.ndarray, jnp.ndarray]: Reordered eigenvalues and eigenvectors.
    """
    orderedIndices = jnp.argsort(eigenvalues, descending=True) 
    orderedEigenvalues = eigenvalues[orderedIndices] 
    orderedEigenvectors = eigenvectors[:, orderedIndices] 

    return orderedEigenvalues, orderedEigenvectors


def svd(data:jnp.ndarray) -> tuple[jnp.ndarray, jnp.ndarray, jnp.ndarray]:
    """ From a data matrix, return its Singular Value Decomposition (SVD) 
    computed using the eigendecomposition of the data's covariance matrix.

    Args:
        data (jnp.ndarray): Rectangular matrix

    Returns:
        tuple[jnp.ndarray, jnp.ndarray, jnp.ndarray]: SVD of the data matrix, where data = U.S.V^T
    """

    # Computing the data matrix's right-singular vectors, as well as the singular values
    eigenvalues, rEigenvectors = jnp.linalg.eig(data.T @ data)
    eigenvalues, rEigenvectors = reorder(eigenvalues, rEigenvectors)
    Vt = jnp.real(rEigenvectors.T)    
    S = jnp.real(eigenvalues ** (1/2))

    # Computing the data matrix's left-singular vectors
    eigenvalues, lEigenvectors = jnp.linalg.eig(data @ data.T)
    eigenvalues, lEigenvectors = reorder(eigenvalues, lEigenvectors)
    U = jnp.real(lEigenvectors)

    
    return U, S, Vt

In [53]:
def pca(data:jnp.ndarray, nbComponents:int, svd=True) -> jnp.ndarray:
    """
    A partir de um conjunto de dados, retorna a matriz de projeção para as k componentes principais.

    Args:
        data (jnp.ndaaray): Dados para reduzir dimensionalidade.
        nbComponents (int): Número de componentes desejadas na projeção.

    Returns:
        jnp.ndarray: Matriz de projeção.
    """

    covarianceMatrix = (1/len(data)) * data.T @ data
    
    if svd:
        # Realizando a decomposição em valores singulares. U terá vetores coluna ortogonais.  
        # A pricípio, o ordenamento não é necessário pois svd() já retorna os valores singulares ordenados.  
        U, S, Ut = jax.scipy.linalg.svd(covarianceMatrix) 
        projectionMatrix = U[:, :nbComponents]
    else:
        # Decomposição a partir de autovalores e autovetores.
        # Aqui, o ordenamento dos autovetores baseado nos autovalores é necessário.
        eigenvalues, eigenvectors = jnp.linalg.eig(covarianceMatrix)  
        indices = jnp.argsort(eigenvalues, descending=True)           
        projectionMatrix = eigenvectors[:, indices]  
        projectionMatrix = projectionMatrix[:, :nbComponents]
   

    return projectionMatrix

In [54]:
def knn(xTrain:jnp.ndarray, yTrain:jnp.ndarray, xTest:jnp.ndarray, k:int, metric:Callable[[jnp.ndarray, jnp.ndarray], float]) -> jnp.ndarray:
    """ Implementa a classificação de um conjunto de dados a partir do algoritmo de K-Nearest Neighbors (KNN).

    Args:
        xTrain (jnp.ndarray): Dados de treino
        yTrain (jnp.ndarray): Labels para os dados de treino
        xTest (jnp.ndarray): Dados de teste
        k (int): Número de vizinhos mais próximos usados para a classificação.
        metric (Callable): Função que calcula distância entre dois pontos.

    Returns:
        jnp.ndarray: Array contendo as predições realizadas para o conjunto de dados de teste xTest.
    """
    
    # Implementação JAX-friendly do cálculo da matriz de distâncias para um conjunto de pontos de teste.
    # O cálculo da matriz é feito vetorizando duas vezes a função de métrica, para calcular dois a dois
    # as distâncias entre pontos de treino e de teste.
    distances = jax.vmap(lambda train_point: jax.vmap(metric, in_axes=(None, 0))(train_point, xTest))(xTrain)

    # Ordenando as distâncias entre pontos, e pegando os targets/labels dos k pontos mais próximos para cada ponto de treino
    sorted_indices = jnp.argsort(distances, axis=0)
    nearestNeighborsIndices = sorted_indices[:k, :]
    nearestNeighbors = yTrain[nearestNeighborsIndices].astype(int)

    # Contagem dos números de cada target presente na lista de vizinhos próximos e definição do target estimado para o ponto
    # de teste baseado numa voto de maioria.
    totalLabels = 3
    targetCounts = jax.vmap(lambda neighbors: jnp.bincount(neighbors, minlength=totalLabels, length=3))(nearestNeighbors.T) 
    most_common_classes = jnp.argmax(targetCounts, axis=1)
    
    return most_common_classes

In [55]:
U1, S1, Vt1 = jax.scipy.linalg.svd(trainData) 
U2, S2, Vt2 = svd(trainData) 

# Os dois arrays devem ser iguais, a menos de um sinal em cada coluna
print("Comparando U: ")
print(U1)
print(U2)

# Os dois arrays devem ser iguais, a menos de um sinal em cada linha
print("\n\nComparando Vt: ")
print(Vt1)
print(Vt2)

# Os valores principais devem ser exatamente iguais
print("\n\nComparando valores principais: ")
print(S1)
print(S2)

Comparando U: 
[[-3.15052629e-01  7.07028687e-01 -4.30500388e-01 -3.75764936e-01
   1.02657732e-03 -8.74909014e-02 -2.57892519e-01 -1.10567575e-02
  -5.09763742e-03 -3.13859642e-03]
 [-3.12000215e-01 -2.74172276e-02  1.04206525e-01  8.48473161e-02
   4.36008036e-01  7.91881680e-01 -2.57892102e-01 -1.10568590e-02
  -5.09756757e-03 -3.13867186e-03]
 [-3.16135943e-01 -2.43040919e-03 -6.51312619e-03  1.17119402e-03
  -7.60586699e-04  2.79027410e-03  4.27988142e-01 -8.14511180e-01
  -2.12080002e-01 -9.17623043e-02]
 [-3.16135943e-01 -2.43036449e-03 -6.51308894e-03  1.17111951e-03
  -7.60556897e-04  2.79049017e-03  3.80615503e-01  5.14487088e-01
  -6.93180859e-01 -9.96775851e-02]
 [-3.16135943e-01 -2.43036449e-03 -6.51310384e-03  1.17108971e-03
  -7.60437688e-04  2.79028155e-03  3.73211622e-01  2.28072390e-01
   6.02946281e-01 -5.87496221e-01]
 [-3.03516477e-01 -6.92516446e-01 -5.01737535e-01 -2.51667321e-01
   1.01275802e-01 -1.90521121e-01 -2.57892549e-01 -1.10567557e-02
  -5.09766536e-03 

In [65]:
nbComponents = 1
# Realizando a decomposição em valores singulares. U terá vetores coluna ortogonais.  
# A pricípio, o ordenamento não é necessário pois svd() já retorna os valores singulares ordenados.  
# U, S, Vt = svd(trainData) 
U, S, Vt = jax.scipy.linalg.svd(trainData)
projectionMatrix = U[:, :nbComponents]
projectionMatrix.shape

zeroRows = U.shape[0] - Vt.shape[0]
zeroColumns = Vt.shape[1]
_zeros = jnp.zeros((zeroRows, zeroColumns))

Sdiag = jnp.vstack([jnp.diag(S), _zeros])

In [66]:
UTrunk = U[:, :nbComponents]
VtTrunk = Vt[:nbComponents, :]
SdiagTrunk = Sdiag[:nbComponents, :nbComponents]

print(UTrunk.shape, SdiagTrunk.shape, VtTrunk.shape)

approximateData = UTrunk @ SdiagTrunk @ VtTrunk
approximateData

(10, 1) (1, 1) (1, 10)


Array([[ 4.3175097,  2.987984 ,  3.9839792,  3.989963 ,  2.987984 ,
        -0.       ,  3.326395 ,  2.4845333,  4.487632 ,  1.9919896],
       [ 4.275679 ,  2.9590347,  3.9453802,  3.951306 ,  2.9590347,
        -0.       ,  3.294167 ,  2.4604616,  4.4441533,  1.9726901],
       [ 4.3323555,  2.998258 ,  3.997678 ,  4.0036826,  2.998258 ,
        -0.       ,  3.337833 ,  2.4930763,  4.5030627,  1.998839 ],
       [ 4.3323555,  2.998258 ,  3.997678 ,  4.0036826,  2.998258 ,
        -0.       ,  3.337833 ,  2.4930763,  4.5030627,  1.998839 ],
       [ 4.3323555,  2.998258 ,  3.997678 ,  4.0036826,  2.998258 ,
        -0.       ,  3.337833 ,  2.4930763,  4.5030627,  1.998839 ],
       [ 4.159417 ,  2.8785741,  3.8380995,  3.8438642,  2.8785741,
        -0.       ,  3.2045937,  2.393558 ,  4.32331  ,  1.9190497],
       [ 4.57785  ,  3.1681554,  4.224208 ,  4.2305527,  3.1681554,
        -0.       ,  3.5269723,  2.6343474,  4.75823  ,  2.112104 ],
       [ 4.3323555,  2.998258 ,  3.997678

In [67]:
diff = jnp.abs(approximateData - trainData)
jnp.sum(diff)

Array(13.399259, dtype=float32)

# Testando a implementação da decomposição em valores principais a partir dos autovalores e autovetores da matriz de covariância dos dados.

Aqui geramos uma matriz aleatória, calculamos seus valores e vetores principais, e comparamos com a implementação nativa do JAX para fins de checagem de sanidade. 

Vemos que a matrix ortogonal de vetores principais não é a mesma obtida pela implementação do JAX, porém isso acontece somente devido a uma inversão do sinal de alguns dos vetores. Nas matrizes U e Ut, isso se traduz respectivamente em uma inversão de sinal de colunas ou linhas especificas, o que não altera as propriedades matemáticas da nossa decomposição. Portanto, a menos de desvios numéricos, temos uma implementação equivalente feita a partir dos autovetores e autovalores da matriz de dados. 

In [86]:
# Gerando os dados aleatórios para teste
key = jax.random.PRNGKey(0)
random_array = jax.random.uniform(key, (5, 4))
data = random_array
# covarianceMatrix = (1/len(data)) * data.T @ data

# Calculando a SVD usando dois métodos diferentes: implementação nativa e usando autovalores e autovetores
U1, S1, Vt1 = jax.scipy.linalg.svd(random_array) 
U2, S2, Vt2 = svd(random_array) 

In [87]:
# Os dois arrays devem ser iguais, a menos de um sinal em cada coluna
print("Comparando U: ")
print(U1)
print(U2)

# Os dois arrays devem ser iguais, a menos de um sinal em cada linha
print("\n\nComparando Vt: ")
print(Vt1)
print(Vt2)

# Os valores principais devem ser exatamente iguais
print("\n\nComparando valores principais: ")
print(S1)
print(S2)

Comparando U: 
[[-0.3816126  -0.44712886  0.16320483 -0.00707924 -0.7923141 ]
 [-0.3532665  -0.64519143  0.30644715  0.09584763  0.59651834]
 [-0.55154705  0.03511621 -0.7935745   0.24159597  0.08020839]
 [-0.434076    0.22940853  0.05902299 -0.86346376  0.09947979]
 [-0.48677325  0.5744059   0.4961955   0.43223238  0.00864178]]
[[ 0.38161245 -0.44712877 -0.16320463 -0.00707943 -0.7923142 ]
 [ 0.35326666 -0.64519167 -0.30644682  0.09584807  0.59651816]
 [ 0.55154705  0.03511646  0.7935746   0.24159612  0.08020833]
 [ 0.43407613  0.22940844 -0.05902323 -0.8634638   0.09948031]
 [ 0.4867733   0.57440597 -0.49619573  0.43223217  0.00864168]]


Comparando Vt: 
[[-0.56254905 -0.45437545 -0.42743266 -0.5425707 ]
 [-0.7061053   0.49096233  0.5045005  -0.07649222]
 [ 0.26226154 -0.4355637   0.71912193 -0.47367397]
 [ 0.34084183  0.6023148  -0.21364602 -0.68949187]]
[[-0.5625492  -0.45437527 -0.42743248 -0.54257065]
 [-0.7061045   0.49096182  0.5045014  -0.07649291]
 [ 0.262262   -0.4355635   0

In [88]:
zeroRows = U1.shape[0] - Vt1.shape[0]
zeroColumns = Vt1.shape[1]
_zeros = jnp.zeros((zeroRows, zeroColumns))

S1diag = jnp.vstack([jnp.diag(S1), _zeros])
S1diag

Array([[2.4590733 , 0.        , 0.        , 0.        ],
       [0.        , 0.93830615, 0.        , 0.        ],
       [0.        , 0.        , 0.73117435, 0.        ],
       [0.        , 0.        , 0.        , 0.5241217 ],
       [0.        , 0.        , 0.        , 0.        ]], dtype=float32)

In [90]:
random_array_composed = U1 @ S1diag @ Vt1
random_array_composed
# print(jnp.sum(random_array - (U1 @ jnp.diag(S1) @ Vt1)))


Array([[0.85417694, 0.16620067, 0.27605486, 0.48728168],
       [0.99204445, 0.03016007, 0.21629447, 0.3768715 ],
       [0.6307006 , 0.9614446 , 0.15203044, 0.9209032 ],
       [0.30555257, 0.29931307, 0.69257087, 0.8542827 ],
       [0.4651739 , 0.7869307 , 0.99605304, 0.28018567]], dtype=float32)

In [93]:
print(jnp.sum(random_array - random_array_composed))
print(random_array_composed)
print(random_array)

-3.3080578e-06
[[0.85417694 0.16620067 0.27605486 0.48728168]
 [0.99204445 0.03016007 0.21629447 0.3768715 ]
 [0.6307006  0.9614446  0.15203044 0.9209032 ]
 [0.30555257 0.29931307 0.69257087 0.8542827 ]
 [0.4651739  0.7869307  0.99605304 0.28018567]]
[[0.85417664 0.16620052 0.27605474 0.48728156]
 [0.9920441  0.03015983 0.21629429 0.37687123]
 [0.63070035 0.96144474 0.15203023 0.92090297]
 [0.30555236 0.29931295 0.6925707  0.8542826 ]
 [0.46517384 0.7869307  0.99605286 0.28018546]]
